In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import random
from scipy import interpolate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import pickle
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
import shap
import time
from numpy import concatenate
Scaler = MinMaxScaler()

# Load training datase

In [54]:
os.chdir('..\\Algal-bloom-prediction-machine-learning\\Trainning data')
all_df = pd.read_csv('Observation_df.csv',sep = ',')
all_df['Date'] = pd.to_datetime(all_df['Date'])

In [8]:
def resample(all_df,sample_interval):
    drop_row = []
    dT = 0
    i = 1
    sample_df = all_df.copy()

    while True:
        dT = (sample_df['Date'].iloc[i]-sample_df['Date'].iloc[i-1])/np.timedelta64(1, 'D')
        if dT<sample_interval: # set the data frequency (over 10, 14, 20 days)
            #print(sample_df['Date'].iloc[i])
            sample_df.drop(sample_df.index.values[i],axis = 0,inplace  =True)
        else:
            i+=1
        if i>=len(sample_df):break
    return sample_df

In [21]:
# Load testing dataset(Daily data)
def read_daily_df(test_yr,features,file):
    Erken_HydMet = pd.read_csv(file,header = 0)
    Erken_HydMet['Date'] = pd.to_datetime(Erken_HydMet['Date'])
    Erken_HydMet['year'] = Erken_HydMet['Date'].apply(lambda y:y.year)
    Erken_HydMet = Erken_HydMet[Erken_HydMet['year']==test_yr]
    return Erken_HydMet

In [25]:
def training_cv(df,features,nutrient,year):
    df = df[df['YY'].isin(year)]
    X_train = df[features]
    y_train = df[nutrient].values

    GBR = GradientBoostingRegressor(random_state=101)
    n_estimators = [int(x) for x in np.arange(40,140,20)]
    max_depth=[int(x) for x in np.arange(5,10,1)]
    learning_rate = [x for x in [0.01,0.1,1]]
    subsample = [x for x in [0.7,0.8,0.9,1]]
    param_grid = {'n_estimators': n_estimators,
                  'max_depth': max_depth,
                  'learning_rate':learning_rate,
                  'subsample':subsample}
    GBR_rs = RandomizedSearchCV(estimator = GBR,param_distributions = param_grid,n_iter = 30,cv = 10,verbose = 0)
    GBR_rs.fit(X_train,y_train)
    GBR.set_params(**GBR_rs.best_params_,random_state=101).fit(X_train,y_train)
    #rmse = -cross_val_score(GBR, X_train, y_train, cv=5,scoring = 'neg_root_mean_squared_error')
    #r2 = cross_val_score(GBR, X_train, y_train, cv=5,scoring = 'r2')
    #print('Training dataset evaluation:')
    #print("RMSE %.2f (+/- %.2f)" % (rmse.mean(), rmse.std()))
    #print("R2 %.2f (+/- %0.2f)" % (r2.mean(), r2.std()))
    pickle.dump(GBR, open("GBR."+nutrient.split('(')[0]+".dat", "wb"))
    

In [26]:
def predict(Erken_Nut,df,features,nutrient,RMSE,R2):
    #load the target GBR model
    GBR = pickle.load(open("GBR."+nutrient.split('(')[0]+".dat", "rb"))
    #predict yhat using test_X
    Erken_Nut[nutrient]=GBR.predict(Erken_Nut[features])
    nutrient_compare = Erken_Nut[['Date',nutrient]].merge(df[['Date',nutrient]],
                                                          how = 'inner',on = 'Date')
    nutrient_compare.columns = ['Date','Prediction','True']
    Date = pd.DataFrame(pd.date_range(start = pd.Timestamp(2017,1,1),
                                  end = pd.Timestamp(2021,1,1)),
                    columns = ['Date'])
    Erken_Nut_gap = Date.merge(Erken_Nut,how = 'left',on = 'Date')

    RMSE.append(mean_squared_error(nutrient_compare['True'],nutrient_compare['Prediction'],squared = False))    
    R2.append(r2_score(nutrient_compare['True'],nutrient_compare['Prediction']))
    return Erken_Nut[['Date',nutrient]],RMSE,R2

In [55]:
# Resample the dataset 
sample_df = resample(all_df,21)
# Specify the features used to make prediction
features = ['Date','month','U','SST','AirT','delT','Humidity','CC','swr(w/m2)','Prec(mm/d)','inflow(m3/s)',
           'Ice_d','days from iceoff','MLD','thermD','W']
year = list(range(2004,2021))

Erken_NOX = pd.DataFrame(columns = ['Date','NOX(mmole/m3)'])
Erken_O2 = pd.DataFrame(columns = ['Date','O2(mmole/m3)'])
Erken_NH4 = pd.DataFrame(columns = ['Date','NH4(mmole/m3)'])
Erken_PO4 = pd.DataFrame(columns = ['Date','PO4(mmole/m3)'])
Erken_TP = pd.DataFrame(columns = ['Date','TP(mmole/m3)'])
Erken_Si = pd.DataFrame(columns = ['Date','Si(mmole/m3)'])
Erken_Chl = pd.DataFrame(columns = ['Date','Chl(mg/m3)'])

NOX_RMSE = []
NOX_R2 = []
O2_RMSE = []
O2_R2 = []
NH4_RMSE = []
NH4_R2 = []
PO4_RMSE = []
PO4_R2 = []
TP_RMSE = []
TP_R2 = []
Si_RMSE = []
Si_R2 = []
Chl_RMSE = []
Chl_R2 = []

In [56]:
for i in range(len(year)):
    start_time = time.time()
    # Specify the training and testing years
    test_yr = year[i]
    training_yr = year.copy()
    training_yr.remove(test_yr)

    # Load the testing daily df
    file = 'Daily_Observation_df.csv'
    testing_daily_df = read_daily_df(test_yr,features,file) 
    
    os.chdir('..\Sample sparsity test\GBR')
    ## Predict NOX
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','U','Humidity','CC','swr(w/m2)','Ice_d','days from iceoff',
                'MLD','W','thermD']
    model = training_cv(sample_df,features,'NOX(mmole/m3)',training_yr)
    NOX_Model,NOX_RMSE,NOX_R2 = predict(testing_daily_df,all_df,features,'NOX(mmole/m3)',NOX_RMSE,NOX_R2)
    Erken_NOX = pd.concat([Erken_NOX,NOX_Model])
    #Erken_Nut = testing_daily_df.merge(NOX_Model,on = 'Date',how = 'left')
    
    ## Predict O2
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','U','Humidity','CC','swr(w/m2)','Ice_d','days from iceoff',
                'MLD','W','thermD']
    model = training_cv(sample_df,features,'O2(mmole/m3)',training_yr)
    O2_Model,O2_RMSE,O2_R2 = predict(testing_daily_df,all_df,features,'O2(mmole/m3)',O2_RMSE,O2_R2)
    Erken_O2 = pd.concat([Erken_O2,O2_Model])
    #Erken_Nut = Erken_Nut.merge(O2_Model,on = 'Date',how = 'left')
    
    ## Predict NH4
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','U','Humidity','CC','swr(w/m2)','Ice_d','days from iceoff',
                'MLD','W','thermD','NOX(mmole/m3)','O2(mmole/m3)']
    model = training_cv(sample_df,features,'NH4(mmole/m3)',training_yr)
    NH4_Model,NH4_RMSE,NH4_R2 = predict(testing_daily_df,all_df,features,'NH4(mmole/m3)',NH4_RMSE,NH4_R2)
    Erken_NH4 = pd.concat([Erken_NH4,NH4_Model])
    #Erken_Nut = Erken_Nut.merge(NH4_Model,on = 'Date',how = 'left')
    
    ## Predict PO4
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','U','Humidity','CC','swr(w/m2)','Ice_d','days from iceoff',
                'MLD','W','thermD','NOX(mmole/m3)','O2(mmole/m3)']
    model = training_cv(sample_df,features,'PO4(mmole/m3)',training_yr)
    PO4_Model,PO4_RMSE,PO4_R2 = predict(testing_daily_df,all_df,features,'PO4(mmole/m3)',PO4_RMSE,PO4_R2)
    Erken_PO4 = pd.concat([Erken_PO4,PO4_Model])
    #Erken_Nut = Erken_Nut.merge(PO4_Model,on = 'Date',how = 'left')
    
    ## Predict TP
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','Ice_d','days from iceoff','U','Humidity','CC','swr(w/m2)',
                'MLD','W','thermD','NOX(mmole/m3)','PO4(mmole/m3)','O2(mmole/m3)']
    model = training_cv(sample_df,features,'TotP(mmole/m3)',training_yr)
    TP_Model,TP_RMSE,TP_R2 = predict(testing_daily_df,all_df,features,'TotP(mmole/m3)',TP_RMSE,TP_R2)
    Erken_TP = pd.concat([Erken_TP,TP_Model])
    #Erken_Nut = Erken_Nut.merge(TP_Model,on = 'Date',how = 'left')

    ## Predict Si
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','Ice_d','days from iceoff','U','Humidity','CC','swr(w/m2)',
                'MLD','W','thermD','NOX(mmole/m3)','PO4(mmole/m3)','O2(mmole/m3)']
    model = training_cv(sample_df,features,'Si(mmole/m3)',training_yr)
    Si_Model,Si_RMSE,Si_R2 = predict(testing_daily_df,all_df,features,'Si(mmole/m3)',Si_RMSE,Si_R2)
    Erken_Si = pd.concat([Erken_Si,Si_Model])
    #Erken_Nut = Erken_Nut.merge(Si_Model,on = 'Date',how = 'left')
    
    ## Predict Chl via the same GBR method as the one used to pre-generate nutrients
    features = ['inflow(m3/s)','AirT','Prec(mm/d)','delT','Ice_d','days from iceoff','U','Humidity','CC','swr(w/m2)',
                'MLD','W','thermD',
                'NOX(mmole/m3)','PO4(mmole/m3)','Si(mmole/m3)', 'TotP(mmole/m3)','NH4(mmole/m3)','O2(mmole/m3)']
    model = training_cv(sample_df,features,'Chl(mg/m3)',training_yr)
    Chl_Model,Chl_RMSE,Chl_R2 = predict(testing_daily_df,all_df,features,'Chl(mg/m3)',Chl_RMSE,Chl_R2)
    Erken_Chl = pd.concat([Erken_Chl,Chl_Model])
    #Erken_Nut = Erken_Nut.merge(Chl_Model,on = 'Date',how = 'left')

    print('Model takes '+str(round((time.time()-start_time)/60))+' min to run')
    os.chdir('../..')
    os.chdir('..\\Algal-bloom-prediction-machine-learning\\Trainning data')

Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run
Model takes 2 min to run


# Output predicted values and evaluating metrics

In [57]:
os.chdir('..\Sample sparsity test\GBR')
Erken_Nut = Erken_NOX.merge(Erken_NH4,
                on='Date',
                how = 'left').merge(Erken_O2,
                                    on='Date',
                                    how = 'left').merge(Erken_PO4,
                                                        on='Date',
                                                        how = 'left').merge(Erken_TP,
                                                                            on = 'Date',
                                                                            how='left').merge(Erken_Si,
                                                                                              on='Date',
                                                                                              how = 'left').merge(Erken_Chl,
                                                                                                                  on='Date',
                                                                                                                  how = 'left')
Erken_Nut.drop('TP(mmole/m3)',axis = 1,inplace = True)
Erken_Nut.to_csv('GBR predicted nutrient_predicted nutrient and Chl.csv',index = False)

RMSE = pd.DataFrame({'NOX':NOX_RMSE,'NH4':NH4_RMSE,'O2':O2_RMSE,'PO4':PO4_RMSE,'TP':TP_RMSE,'Si':Si_RMSE,'Chl':Chl_RMSE},index = year)
RMSE.to_csv('RMSE.csv',index = False)

R2 = pd.DataFrame({'NOX':NOX_R2,'NH4':NH4_R2,'O2':O2_R2,'PO4':PO4_R2,'TP':TP_R2,'Si':Si_R2,'Chl':Chl_R2},index = year)
R2.to_csv('R2.csv',index = False)